Key ID:  
458744584072543631 - not actived  
719635943846940421 - ЛОЦМАН:КБ 2022,HSL,сетвр  
834732166184233058 - ОЭ КОМПАC-3D v20,HSL,сетвр  
982786152807025115 - ЛОЦМАН:КБ 2021,HSL,сетвр  

Productid:  
3197 ЛОЦМАН:КБ 2022,HSL,сетвр   
3258 КОМПАC-3D v20,HSL,сетвр 

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
pd.options.display.max_colwidth = 100

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Читаем данные с логами:

In [4]:
def merge_logs(month, day):

  for i in range(1, day):

    # чтение файла
    df = pd.read_csv(f"/content/drive/My Drive/projects/determination_average_number_lodsman_users/logs/2021_{month:02d}_{i:02d}_access.log", 
                    sep=r' ', skiprows=1, error_bad_lines=False, warn_bad_lines=False) 
    # задаем признаки
    df.columns = ['nan',	'nan1',	'date', 'time',	'ip',	'user',	'port',	'api' ,	'features' ,	'result' ,	'nan2']
    # уберем лишнии признаки
    df.drop(['nan', 'nan1', 'nan2'], axis=1, inplace=True)
    # удалим пропущенные значения
    df = df.dropna(subset=['user', 'features'])

    # проведем логическую индексацию
    mask = df["user"].str.contains(r"Администратор")
    dff = df.loc[mask]
    mask = dff["features"].str.contains(r"productid=3197")
    dff = dff.loc[mask]


  # соединим логи в одну таблицу
    if i == 1: 
      df_null = pd.DataFrame([np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN]).T
      df_null.columns = ['median_time', 'haspid', 'productid', 'feat', 'sess', 'api', 'span', 'dtype' ]

      df_null = pd.concat([df_null, dff], ignore_index=True)
    else:

      df_null = pd.concat([df_null, dff], ignore_index=True)  

  # уберем лишние признаки и пропущенные значения из созданного переходного датафрейма
  df_null.drop(['median_time', 'haspid', 'productid', 'feat', 'sess', 'api', 'span', 'dtype'], axis=1, inplace=True) 
  df_null = df_null.dropna()
  return df_null

Сделаем из признака `'features'` датафрейм:

In [5]:
def split_features(data):
  for i in range(len(data)-1):
    try:
      if i == 0: 
        zz555 = pd.DataFrame([np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN]).T
        zz555.columns = ['log', 'haspid', 'productid', 'feat', 'sess', 'api', 'span', 'dtype' ]
        zz = pd.DataFrame(str(data["features"][i:i+1]).split(',')).T
        zz.columns = ['log', 'haspid', 'productid', 'feat', 'sess', 'api', 'span', 'dtype' ]
        zz2 = pd.DataFrame(str(data["features"][i+1:i+2]).split(',')).T
        zz2.columns = ['log', 'haspid', 'productid', 'feat', 'sess', 'api', 'span', 'dtype' ]
        zz555 = pd.concat([zz555, zz, zz2], ignore_index=True)
      else:
        zz = pd.DataFrame(str(data["features"][i:i+1]).split(',')).T
        zz.columns = ['log', 'haspid', 'productid', 'feat', 'sess', 'api', 'span', 'dtype' ]
        zz2 = pd.DataFrame(str(data["features"][i+1:i+2]).split(',')).T
        zz2.columns = ['log', 'haspid', 'productid', 'feat', 'sess', 'api', 'span', 'dtype' ]
        zz555 = pd.concat([zz555, zz, zz2], ignore_index=True)
    except: # если будет проблемная строка
      i += 1

  zzz5556 = zz555.drop_duplicates().dropna().reset_index()   # датафрейм из признака 
  return zzz5556

Соединим наши датафреймы с признаком `'features'` и с логами. Уберем лишние признаки, пропуски:

In [6]:
def merge_logs_features(data1, data2):
  df = data1.merge(data2, how='outer', left_index=True, right_index=True)
  df.drop(['port', 'features', 'result', 'index', 'span', 'dtype'], axis=1, inplace=True)
  df = df.dropna().reset_index()
  return df

Прочитаем данные за октябрь:

In [7]:
oct = merge_logs(10, 32)
oct_feat = split_features(oct)
df_oct = merge_logs_features(oct, oct_feat)

Прочитаем данные за ноябрь:

In [8]:
nov = merge_logs(11, 31)
nov_feat = split_features(nov)
df_nov = merge_logs_features(nov, nov_feat)

Прочитаем данные за декабрь:

In [9]:
dec = merge_logs(12, 32)
dec_feat = split_features(dec)
df_dec = merge_logs_features(dec, dec_feat)

Соединим все данные вместе и выведем информацию на экран:

In [10]:
df = pd.concat([df_oct, df_nov, df_dec], ignore_index=True)

Вывыдем последние 5 строк:

In [11]:
df.tail()

,index,date,time,ip,user,log,haspid,productid,feat,sess,api
7404,361,0}2021-12-29,11:44:48,192.168.0.158:62833,Администратор@SRV-KB,363 LOGIN_EX(lm=local,haspid=719635943846940421,productid=3197,feat=622,sess=000001E9,api=7.50
7405,362,0}2021-12-29,11:59:07,192.168.0.158:62915,Администратор@SRV-KB,364 LOGOUT(lm=local,haspid=719635943846940421,productid=3197,feat=622,sess=000001E9,duration=7
7406,363,0}2021-12-29,14:17:39,192.168.0.158:63737,Администратор@SRV-KB,365 LOGOUT(lm=local,haspid=719635943846940421,productid=3197,feat=622,sess=000001DD,duration=12271
7407,364,0}2021-12-29,14:17:46,192.168.0.158:63737,Администратор@SRV-KB,366 LOGOUT(lm=local,haspid=719635943846940421,productid=3197,feat=622,sess=000001C1,duration=20529
7408,365,0}2021-12-29,14:50:16,192.168.0.158:63922,Администратор@SRV-KB,367 LOGOUT(lm=local,haspid=719635943846940421,productid=3197,feat=622,sess=000001C4,duration=27022


Удалим лишние признаки:

In [12]:
df.drop(['index', 'haspid', 'productid'], axis=1, inplace=True)

Приведем столбец `'log'` к нормальному виду:

In [13]:
def replace_bad_space(row):
  text = re.sub(r'\w+\ ', '', row)
  return text

In [14]:
df['log'] = df['log'].apply(replace_bad_space)

In [15]:
df['log'] = df['log'].str.split('(').str[0].add('')

In [16]:
df['log'].value_counts()

   LOGIN_EX    3742
   LOGOUT      3667
Name: log, dtype: int64

In [17]:
df['log'] = df['log'].replace('   LOGIN_EX', 'login')
df['log'] = df['log'].replace('   LOGOUT', 'logout')

Приведем признаки с датами и временем к нужнемому типу:

In [18]:
def replace_bad_symbols(row):
  text = re.sub(r'\w+\}', '', row)
  return text

In [19]:
df['date'] = df['date'].apply(replace_bad_symbols)

In [20]:
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')

In [21]:
df['time'] = pd.to_datetime(df['time'], format='%H:%M:%S')

Создадим столбец с переводом времени в секунды:

In [22]:
df['duration'] = (df['time'].dt.hour*60+df['time'].dt.minute)*60 + df['time'].dt.second

In [34]:
df['time'] = (pd.to_datetime(df['time'], format='%H:%M:%S')).dt.time

Выведем информацию о получившийся таблице:

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7409 entries, 0 to 7408
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      7409 non-null   datetime64[ns]
 1   time      7409 non-null   datetime64[ns]
 2   ip        7409 non-null   object        
 3   user      7409 non-null   object        
 4   log       7409 non-null   object        
 5   feat      7409 non-null   object        
 6   sess      7409 non-null   object        
 7   api       7409 non-null   object        
 8   duration  7409 non-null   int64         
 9   time_hms  7409 non-null   object        
dtypes: datetime64[ns](2), int64(1), object(7)
memory usage: 579.0+ KB


In [35]:
df.head()

,date,time,ip,user,log,feat,sess,api,duration,time_hms
0,2021-10-01,00:40:58,192.168.0.158:59813,Администратор@SRV-KB,login,feat=594,sess=000003F3,api=7.50,2458,00:40:58
1,2021-10-01,05:03:18,192.168.0.158:60775,Администратор@SRV-KB,logout,feat=594,sess=000003F3,duration=1,18198,05:03:18
2,2021-10-01,05:03:18,192.168.0.158:60775,Администратор@SRV-KB,login,feat=594,sess=00000404,api=7.50,18198,05:03:18
3,2021-10-01,08:03:42,192.168.0.158:61435,Администратор@SRV-KB,logout,feat=594,sess=00000404,duration=0,29022,08:03:42
4,2021-10-01,08:03:43,192.168.0.158:61435,Администратор@SRV-KB,login,feat=594,sess=00000406,api=7.50,29023,08:03:43


Создадим сводную таблицу по датам, в которую положим индификатор сессии, дату, продолжительность сессии:

In [69]:
#df_sample = df[df['date'] == '2021-10-01']
df_sample = df

In [70]:
df_group_sess = df_sample.pivot_table(index=['sess', 'date'], values='duration', aggfunc=['first', 'last']).reset_index()

In [71]:
df_group_sess = df_group_sess.droplevel(level=1, axis=1)

In [72]:
df_group_sess['duration'] = df_group_sess['last'] - df_group_sess['first']

Выберем сессии продолжительностью более чем 2 минуты:

In [73]:
df_group_sess = (df_group_sess[df_group_sess['duration'] > 120]).reset_index()

In [74]:
df_group_sess = df_group_sess.drop(['index', 'first', 'last'], axis=1)

Проведем группировку по датам, и создадим признаки количество сессий, продолжительность, среднеяя, медианна:

In [79]:
df_group_sess_days = df_group_sess.pivot_table(index='date', values=['duration'], aggfunc=['count', 'sum', 'mean', 'median']).reset_index()

In [80]:
df_group_sess_days = df_group_sess_days.droplevel(level=1, axis=1)

Посмотрим на таблицу:

In [82]:
df_group_sess_days.head()

,date,count,sum,mean,median
0,2021-10-01,9,53122,5902.444444,2915.0
1,2021-10-02,1,19680,19680.000000,19680.0
2,2021-10-04,13,69754,5365.692308,979.0
3,2021-10-05,13,52593,4045.615385,1351.0
4,2021-10-06,18,70276,3904.222222,1407.5
